 6) Is it possible to train distributed representations for unstructured data (table dataset with categorical features)? How?  
 Для этой задачи можно использовать Word2vec (CBOW, skip-grams) и Glove
Формулировка задачи обучения (функция потерь / функционал качества)
Задача состоит в том, чтобы представить слова корпуса в виде векторов одинаковой размерности в векторном пространстве. При этом векторные представления (embeddings) слов, которые встречаются в похожих контекстах, должны быть близко друг к другу. 
Обучаться можно для каждого слова на двух признаковых описаниях: Continuous bag of words (CBOW, просто встречаемости слов в окне определённого размера вокруг слова) или skip-grams (n-граммы с пропусками, т.е. все слова n-граммы стоят не дальше определённого расстояния друг от друга). 

 
 
 
 
 
 

In [15]:
import gensim
import gensim.downloader as api
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import re

import torch
import torch.nn as nn
import torch.nn.functional as F

from torch.utils.data import DataLoader, TensorDataset
from torchtext.data import Field, TabularDataset, Iterator

from itertools import combinations
from scipy.ndimage.filters import gaussian_filter1d
from tqdm import tqdm, tqdm_notebook

from nltk.stem import WordNetLemmatizer 
from nltk.tokenize import word_tokenize
from nltk.tokenize import sent_tokenize, word_tokenize 

In [4]:
df_train = pd.read_csv('train.csv')
df_test = pd.read_csv('test.csv')

df_train['target'] = (df_train['target'] == 5).astype(np.int)
df_test['target'] = (df_test['target'] == 5).astype(np.int)

df_train.shape

(48192, 3)

In [8]:
df_train

,review,title,target
0,"The staff was very friendly, the breakfast ver...",Walker Gem,1
1,Excellent service - very approachable and prof...,Excellent Service,0
2,Really a top notch place to spend a day at the...,"Good location, warm and friendly staff",1
3,"a little noisy, there was a false fire alarm a...","nice hotel,",0
4,Place had too many animals and I'm allergic to...,Experience,0
...,...,...,...
48187,"A friend of mine always books the cheapest, ba...","Comfy, cozy but oh, so grand!",0
48188,Stayed here with my family over Spring Break i...,Great location and price for family lodging.,0
48189,One word AWFUL and the pool was closed,Good hotel in a quiet part of Memphis,0
48190,Never will stay here again. Dirty towels shirt...,Filthy,0


In [8]:
import nltk
import numpy as np
import pandas as pd
from nltk.corpus import wordnet as wn
from nltk.stem.wordnet import WordNetLemmatizer
from nltk import word_tokenize, pos_tag
from sklearn import metrics
import sys
import warnings
warnings.filterwarnings('ignore')

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Dense , Input , LSTM , Embedding, Dropout , Activation, GRU, Flatten
from keras.layers import Bidirectional, GlobalMaxPool1D
from keras.models import Model, Sequential
from keras.layers import Convolution1D
from keras import initializers, regularizers, constraints, optimizers, layers

import re


SEED = 42
np.random.seed(SEED)

Using TensorFlow backend.


In [9]:
import torch
import torch.nn as nn
import torch.nn.functional as F

from torch.utils.data import DataLoader, TensorDataset

In [10]:
from collections import defaultdict

In [12]:
!pip install tensorflow

     |████████████████████████████████| 120.8MB 4.6MB/s eta 0:00:01    |███▌                            | 13.2MB 328kB/s eta 0:05:28     |███▉                            | 14.6MB 240kB/s eta 0:07:21     |█████████████████████████▌      | 96.3MB 1.8MB/s eta 0:00:14     |██████████████████████████▏     | 98.6MB 1.8MB/s eta 0:00:13
     |████████████████████████████████| 3.9MB 4.7MB/s eta 0:00:01
     |████████████████████████████████| 450kB 1.1MB/s eta 0:00:01
     |████████████████████████████████| 2.5MB 259kB/s eta 0:00:01
     |████████████████████████████████| 28.4MB 3.3MB/s eta 0:00:01
     |████████████████████████████████| 112kB 10.6MB/s eta 0:00:01
     |████████████████████████████████| 1.3MB 8.4MB/s eta 0:00:01
     |████████████████████████████████| 61kB 14.7MB/s eta 0:00:01
     |████████████████████████████████| 71kB 11.3MB/s eta 0:00:01
     |████████████████████████████████| 81kB 11.3MB/s eta 0:00:01
     |████████████████████████████████| 92kB 7.3MB/s eta 0:00:011
     |█

In [14]:
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/Asalamatina/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/Asalamatina/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/Asalamatina/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package punkt to
[nltk_data]     /Users/Asalamatina/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [11]:
stop_words=nltk.corpus.stopwords.words('english')

In [12]:
lemmatizer = WordNetLemmatizer()

In [13]:
nltk.download('wordnet')

nltk.download('wordnet')
def tokenizer(text):
    return word_tokenize(text)



[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/Asalamatina/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/Asalamatina/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [25]:
tag_map = defaultdict(lambda : wn.NOUN)
tag_map['J'] = wn.ADJ
tag_map['V'] = wn.VERB
tag_map['R'] = wn.ADV
tag_map['S'] = wn.ADJ_SAT

lmtzr = WordNetLemmatizer()

def lemmatize(tokens):
  lemmas = []
  for token, tag in pos_tag(tokens):
      lemmas.append(lmtzr.lemmatize(token, tag_map[tag[0]]))
  return lemmas

In [28]:
def lemmatizer(text):
  return lemmatize(tokenizer(text))

In [31]:
device =  torch.device('cpu')

In [111]:
TEXT = Field(include_lengths=True, batch_first=True, 
             tokenize=tokenizer,
             eos_token='<eos>',
             lower=True,
             stop_words=nltk.corpus.stopwords.words('english'))
LABEL = LabelField(dtype=tt.int64, use_vocab=True, preprocessing=lambda x: classes[x])

test_dataset = TabularDataset('test.csv', format='csv',
                              fields = [(None,None),(None,None),('review', TEXT),('target', LABEL),(None,None)],
                              skip_header=True)

train_dataset = TabularDataset('train.csv', format='csv',
                              fields = [(None,None),(None,None),('review', TEXT),('target', LABEL),(None,None)],
                              skip_header=True)

In [112]:
 TEXT.build_vocab(df_train)

In [113]:
LABEL.build_vocab(df_train)

In [117]:
train_dataset.build_vocab

<generator object Dataset.__getattr__ at 0x1a596b4dd0>

In [76]:
class MyModel(nn.Module):
    
    def __init__(self, vocab_size, embed_size, hidden_size, kernels):
        super(MyModel, self).__init__()
        self.embedding = nn.Embedding(vocab_size, embed_size)
        
        self.convs = nn.ModuleList([nn.Conv1d(embed_size, hidden_size, k, padding=5) for k in kernels])
        
        self.fc = nn.Linear(hidden_size * len(kernels), 3)
        
    def forward(self, x):
        
        x = self.embedding(x)
        x = x.transpose(1,2)
        
        concatenated = []
        for conv in self.convs:
            z = conv(x)
            z = F.avg_pool1d(z, kernel_size=z.size(2))
            z = z.squeeze(2)
            concatenated.append(z)
            
        x = torch.cat(concatenated, 1)
        x = self.fc(x)
        return x

In [77]:
def early_stopping(loss, patience=5):
    if len(loss) < patience:
        return False
    best = np.min(np.array(loss))
    return all(map(lambda x: x > best, loss[-patience:]))

def train_batch(model, batch_x, batch_y):
    model.train()
    optimizer.zero_grad()
    pred = model(batch_x)
    loss = criterion(pred, batch_y)
    loss.backward()
    optimizer.step()
    return loss.data.detach().item()

def eval_batch(model, batch_x, batch_y):
    model.eval()
    
    with torch.no_grad():
        pred = model(batch_x)
        loss = criterion(pred, batch_y)
    return loss.data.item()

def train_model(model, train_loader, valid_loader, scheduler, n_epochs, early_stopping_patience=5):
    history = {'train': [], 'valid': []}    
    for epoch in range(n_epochs):
        epoch_train_loss = 0
        for batch in tqdm(train_loader, desc=f'epoch:{epoch}', leave=True):

            loss = train_batch(model, batch.text[0], batch.label)
            epoch_train_loss += loss
            if scheduler:
                scheduler.step(loss)

        # average loss for epoch
        epoch_train_loss /= len(train_loader)
        history['train'].append(epoch_train_loss)
        
        epoch_valid_loss = 0
        for batch in valid_loader:
            loss = eval_batch(model, batch.text[0], batch.label)
            epoch_valid_loss += loss
            
        epoch_valid_loss /= len(valid_loader)
        print(f'train loss: {epoch_train_loss:.3f} valid loss:{epoch_valid_loss:.3f}')
        history['valid'].append(epoch_valid_loss)
        
        if early_stopping(history['valid'], patience=5):
            print('early stopping')
            break

    print('history valid:', history['valid'])

    return history

In [118]:
batch_size = 32

model = MyModel(len(TEXT.vocab.itos),
                embed_size=100,
                hidden_size=128,
                kernels=[2,3,4,5]
               )

train_iterator, valid_iterator, test_iterator = BucketIterator.splits(
    (train, valid, testdataset),
    batch_sizes=(batch_size, batch_size, batch_size),
    shuffle=True,
    sort_key=lambda x: len(x.text),
#     sort_within_batch=True,
)

optimizer = optim.Adam(model.parameters())
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, patience=5, verbose=True, cooldown=5)
criterion = nn.CrossEntropyLoss()

history = train_model(model, train_iterator, valid_iterator, scheduler, 3, 2)
test_model(model, test_iterator)


NameError: name 'train' is not defined

In [14]:

stop_words = set(stopwords.words("english"))  
lemmatizer = WordNetLemmatizer()

def preprocessing(sentens):
  description = re.sub("[^a-zA-Z]"," ",sentens)
  description = description.lower()
  description = [lemmatizer.lemmatize(token) for token in description.split(" ")]
  description = [lemmatizer.lemmatize(token, "v") for token in description]
  description = [word for word in description if not word in stop_words]
  description = " ".join(description)
  return description

description_list = []
for sentens in tqdm.tqdm_notebook(df['review'].values):
  description_list.append(preprocessing(sentens))

NameError: name 'stopwords' is not defined

In [5]:
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer(max_features=5000)
vectorizer.fit(df_train['text'].values)

KeyError: 'text'